In [225]:
import pandas as pd

In [226]:
#Load all dataset
appearances_df = pd.read_csv('appearances.csv')
players_df = pd.read_csv('players.csv')
clubs_df = pd.read_csv('clubs.csv')
valuations_df = pd.read_csv('player_valuations.csv')
clubs_df.head()

,club_id,name,pretty_name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,url
0,1032,fc-reading,Fc Reading,GB1,33.66,26,25.9,12,46.2,6,Select Car Leasing Stadium,24161,+£8.37m,Brian McDermott,https://www.transfermarkt.co.uk/fc-reading/sta...
1,2323,orduspor,Orduspor,TR1,NaN,0,NaN,0,NaN,0,19 Eylül Stadyum,11024,+-0,Héctor Cúper,https://www.transfermarkt.co.uk/orduspor/start...
2,1387,acn-siena-1904,Acn Siena 1904,IT1,4.32,30,26.2,6,20.0,2,Artemio Franchi,15373,£-6Th.,Serse Cosmi,https://www.transfermarkt.co.uk/acn-siena-1904...
3,3592,kryvbas-kryvyi-rig,Kryvbas Kryvyi Rig,UKR1,2.95,24,24.5,2,8.3,0,Shakhty Zhovtneva,2500,£-270Th.,Oleg Taran,https://www.transfermarkt.co.uk/kryvbas-kryvyi...
4,1071,wigan-athletic,Wigan Athletic,GB1,12.38,29,26.5,14,48.3,6,DW Stadium,25133,£-140Th.,Roberto Martínez,https://www.transfermarkt.co.uk/wigan-athletic...


In [227]:
# clean table appearances
# drop columns and calculate players' total goals, assists,
# minutes_played, yellow cards and red cards of each year
appearances_df = appearances_df.drop(['player_pretty_name', 'game_id', 'appearance_id', 'competition_id', 'player_club_id'], axis=1)
appearances_df['date'] = pd.to_datetime(appearances_df['date'])
appearances_df= appearances_df.groupby(by=[appearances_df.player_id, appearances_df.date.dt.year]).sum().reset_index()
appearances_df.head()

/var/folders/25/v0klc1td0jb3lh9rsz8kf4cm0000gn/T/ipykernel_60657/763911003.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  appearances_df= appearances_df.groupby(by=[appearances_df.player_id, appearances_df.date.dt.year]).sum().reset_index()


,player_id,date,goals,assists,minutes_played,yellow_cards,red_cards
0,10,2014.0,4,3,496,2,0
1,10,2015.0,12,8,2429,6,0
2,10,2016.0,8,5,1078,1,0
3,26,2014.0,0,0,1530,0,0
4,26,2015.0,0,0,2160,0,0


In [228]:
#clean table clubs
#drop columns and clean the missing value
clubs_df = clubs_df[['club_id','total_market_value']]
clubs_df['total_market_value'] = clubs_df['total_market_value'].fillna(1)
clubs_df.head()

,club_id,total_market_value
0,1032,33.66
1,2323,1.00
2,1387,4.32
3,3592,2.95
4,1071,12.38


In [229]:
# clean table players
# drop columns and clean the missing value
players_df = players_df[['player_id', 'country_of_birth', 'country_of_citizenship',
                         'date_of_birth', 'position', 'sub_position', 'foot', 'height_in_cm', 'club_id']]
players_df = players_df.dropna()
# convert the types of column
players_df['date_of_birth'] = pd.DatetimeIndex(pd.to_datetime(players_df['date_of_birth'])).year
players_df['date_of_birth'] = players_df['date_of_birth'].astype(int)
# assign integer values for foot, position, sub position and countries
players_df['foot'] = players_df['foot'].map({'Right': 1, 'Left': 0, 'Both': 2})
players_df['position'] = players_df['position'].map({'Attack': 0, 'Defender': 1, 'Midfield': 2, 'Goalkeeper': 3})
countries_of_birth = set(players_df['country_of_birth'])
countries_of_citizenship = set(players_df['country_of_citizenship'])
countries = countries_of_birth.union(countries_of_citizenship)
country_dict = {country:val for val, country in enumerate(countries)}
players_df['country_of_birth'] = players_df['country_of_birth'].map(country_dict)
players_df['country_of_citizenship'] = players_df['country_of_citizenship'].map(country_dict)
sub_position = set(players_df['sub_position'])
sp_dict = {position:val for val, position in enumerate(sub_position)}
players_df['sub_position'] = players_df['sub_position'].map(sp_dict)
print(players_df.head())

   player_id  country_of_birth  country_of_citizenship  date_of_birth  \
0     254016                62                      62           1992   
2      31451               156                     156           1986   
3      44622                60                      43           1981   
4      30802               141                     141           1984   
5     214776                33                      33           1993   

   position  sub_position  foot  height_in_cm  club_id  
0         2             6     1           186      855  
2         0             4     1           192       23  
3         1            11     1           182     3691  
4         0             4     1           183     3302  
5         0             4     1           191       43  


In [230]:
# clean table players valuation
# drop columns and calculate average market value of each year
valuations_df = valuations_df[['player_id', 'date', 'market_value']]
valuations_df['date'] = pd.to_datetime(valuations_df['date'])
valuations_df = valuations_df.groupby(by=[valuations_df.player_id, valuations_df.date.dt.year]).mean().reset_index()

/var/folders/25/v0klc1td0jb3lh9rsz8kf4cm0000gn/T/ipykernel_60657/2020395686.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  valuations_df = valuations_df.groupby(by=[valuations_df.player_id, valuations_df.date.dt.year]).mean().reset_index()


In [231]:
# merge all tables into one
# export to a csv file
df1 = players_df.merge(clubs_df)
df2 = appearances_df.merge(df1)
df = df2.merge(valuations_df)
df.head()
df.to_csv('final.csv')